In [1]:
import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
import random
import os
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve

In [2]:
data_reader=ReadData('/root/autodl-fs/hallscope/pope/popular/train')
data=data_reader.read_all()
random.shuffle(data)

In [3]:
df=pd.DataFrame(data)

In [4]:
df.head()

,img_path,question,label,question_id,category,most_likely
0,[/root/autodl-tmp/coco_images/train/COCO_train...,[Is there a suitcase in the image?\nAnswer the...,"[1, 0, 1, 0, 1, 0, 1, 0]","[tensor(5017), tensor(5018), tensor(5019), ten...","[popular, popular, popular, popular, popular, ...",{'embedding': [[[ 0.00389099 0.00149536 0.00...
1,[/root/autodl-tmp/coco_images/train/COCO_train...,[Is there a bus in the image?\nAnswer the ques...,"[1, 0, 1, 0, 1, 0, 1, 0]","[tensor(1625), tensor(1626), tensor(1627), ten...","[popular, popular, popular, popular, popular, ...",{'embedding': [[[ 0.00389099 0.00149536 0.00...
2,[/root/autodl-tmp/coco_images/train/COCO_train...,[Is there a knife in the image?\nAnswer the qu...,"[1, 0, 1, 0, 1, 0, 1, 0]","[tensor(7897), tensor(7898), tensor(7899), ten...","[popular, popular, popular, popular, popular, ...",{'embedding': [[[ 0.00389099 0.00149536 0.00...
3,[/root/autodl-tmp/coco_images/train/COCO_train...,[Is there a person in the image?\nAnswer the q...,"[1, 0, 1, 0, 1, 0, 1, 0]","[tensor(6649), tensor(6650), tensor(6651), ten...","[popular, popular, popular, popular, popular, ...",{'embedding': [[[ 0.00389099 0.00149536 0.00...
4,[/root/autodl-tmp/coco_images/train/COCO_train...,[Is there a truck in the image?\nAnswer the qu...,"[1, 0, 1, 0, 1, 0, 1, 0]","[tensor(2745), tensor(2746), tensor(2747), ten...","[popular, popular, popular, popular, popular, ...",{'embedding': [[[ 0.00389099 0.00149536 0.00...


In [15]:
embeddings=df['most_likely'].apply(lambda x: x['embedding']).to_list()

In [16]:
embeddings=np.concatenate(embeddings,axis=0)

In [17]:
embeddings.shape

(20000, 33, 4096)

In [18]:
gt_label=df['label'].to_list()
gt_label=np.concatenate(gt_label,axis=0)

In [19]:
gt_label.shape

(20000,)

In [33]:
length=len(gt_label)
train_rate=0.8

all_indices = np.random.permutation(length)
train_val_idxs = all_indices[:int(
    train_rate * length)]  # trainset and validation index

test_idxs=all_indices[int(
    train_rate * length):]  #test index

validset_len = 3000
# exclude validation samples.
train_idxs = train_val_idxs[:len(
    train_val_idxs) - validset_len]  # trainset index
val_idxs = train_val_idxs[len(
    train_val_idxs) - validset_len:]  # validation index
gt_label_test = []
gt_label_wild = []
gt_label_val = []

'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
gt_label_test = gt_label[test_idxs]
gt_label_wild = gt_label[train_idxs]
gt_label_val = gt_label[val_idxs]

print(f'trainset length:{len(gt_label_wild)}')
print(f'validset length:{len(gt_label_val)}')
print(f'testset length:{len(gt_label_test)}')
print(f'trainset postive: {sum(gt_label_wild==1)} negative: {sum(gt_label_wild==0)}')
print(
    f'testset postive: {sum(gt_label_test==1)} negative: {sum(gt_label_test==0)}')

trainset length:13000
validset length:3000
testset length:4000
trainset postive: 6482 negative: 6518
testset postive: 2008 negative: 1992


In [34]:
all_embeddings = embeddings
all_embeddings.shape

(20000, 33, 4096)

In [35]:
embedding_train = all_embeddings[train_idxs][:, 1:, :]
embedding_val = all_embeddings[val_idxs][:, 1:, :]
embedding_test = all_embeddings[test_idxs][:, 1:, :]

In [36]:
%load_ext autoreload 
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCADiscriminator
from ml_utils.metrics import auroc,get_best_split_from_scores

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Grid Search for Best 'n_components' & 'layer'

In [37]:
def evaluator(n_components,i_layer,X,y):
    discriminator=PCADiscriminator(n_components,X[:,i_layer,:])
    scores=discriminator.get_score()
    return auroc(scores,y)

In [38]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,12),
    'layer':range(embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,gt_label_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer


  0%|          | 0/11 [00:00<?, ?it/s]

found best n_components:1,layer:0 Score: 0.513694164185075


found best n_components:1,layer:16 Score: 0.5400297791012934
found best n_components:1,layer:17 Score: 0.8722978799057735
found best n_components:1,layer:20 Score: 0.8896697631005822
found best n_components:1,layer:21 Score: 0.9060989377305659
found best n_components:1,layer:22 Score: 0.9299888883950398
found best n_components:1,layer:26 Score: 0.9379616871860972
found best n_components:1,layer:27 Score: 0.944389083959287


  9%|▉         | 1/11 [00:05<00:52,  5.22s/it]

...

 82%|████████▏ | 9/11 [00:47<00:10,  5.30s/it]

found best n_components:10,layer:29 Score: 0.951532957020312


 91%|█████████ | 10/11 [00:52<00:05,  5.31s/it]

found best n_components:11,layer:28 Score: 0.9516774078847949


100%|██████████| 11/11 [00:58<00:00,  5.30s/it]


In [39]:
best_paras

{'n_components': 11, 'layer': 28, 'best_score': np.float64(0.9516774078847949)}

In [40]:
best_layer=best_paras['layer']

View Results on Training set and Test set

In [41]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(gt_label_val)

In [42]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_train[:,best_layer,:])
scores=discriminator.get_score()
y_train=(scores>best_split)
print(f'train set acc: {accuracy_score(gt_label_wild,y_train)} AUROC:{roc_auc_score(gt_label_wild,scores)}')

train set acc: 0.6935384615384615 AUROC:0.950703444447716


In [43]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_test[:,best_layer,:])
scores=discriminator.get_score()
preds=(scores>best_split)
print(f'test set acc: {accuracy_score(gt_label_test,preds)} AUROC:{roc_auc_score(gt_label_test,scores)}')

test set acc: 0.8685 AUROC:0.9478101649626396


Train Linear Prob to Get Better Results

In [44]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=gt_label_test

In [45]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [46]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores_test = torch.sigmoid(output).cpu().data.numpy()
pred_test=scores_test>0.5
print(f'test set acc: {accuracy_score(y_test,pred_test)} AUROC:{roc_auc_score(y_test,scores_test)}')

test set acc: 0.6975 AUROC:0.7500340005440087
